In [1]:
### Make Your Own Crypto ETF using Alpaca’s Crypto API

In [18]:
#Imports for Analysis and MC Simulation Program:
from pathlib import Path
import pandas as pd
import pandas_datareader as pdr
import hvplot.pandas
import os
import requests
import json
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

In [19]:
load_dotenv() 

True

In [20]:
from alpaca_trade_api.rest import REST, TimeFrame

API_KEY = ("alpaca_api_key")
SECRET_KEY = ("alpaca_secret_key")

alpaca_client = REST(API_KEY, SECRET_KEY, 'http://paper-api.alpaca.markets')
display(type(API_KEY))
display(type(SECRET_KEY))

str

str

In [21]:
# Set Alpaca API key and secret by calling the os.getenv function and referencing the environment variable names
# Set each environment variable to a notebook variable of the same name
#alpaca_api_key = os.getenv("ALPACA_API_KEY")
#alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Check the values were imported correctly by evaluating the type of each
#display(type(alpaca_api_key))
#display(type(alpaca_secret_key)) 

In [22]:
# Create your Alpaca API REST object by calling Alpaca's tradeapi.REST function
# Set the parameters to your alpaca_api_key, alpaca_secret_key and api_version="v2" 
#alpaca = tradeapi.REST(
#    alpaca_api_key,
#    alpaca_secret_key,
#    api_version = "v2") 

In [23]:
limit_rows = 10000
bars = alpaca_client.get_crypto_bars("BTCUSD", TimeFrame.Day, "2020-01-01", "2022-02-01").df 

In [24]:
# Use the Alpaca get_bars function to gather the price information for each ticker
# Include the function parameters: tickers, timeframe, start, end, and limit 
# limit_rows = 10000
# Be sure to call the df property to ensure that the returned information is set as a DataFrame
#crpto_df = alpaca_client.get_bars('BTCUSD', TimeFrame.Day, '2020-01-01', '2022-02-01').df

HTTPError: 403 Client Error: Forbidden for url: https://data.alpaca.markets/v2/stocks/BTCUSD/bars?timeframe=1Day&adjustment=raw&start=2020-01-01&end=2022-02-01

In [ ]:
returns = {}

for coin in coins:
  bars = rest.get_crypto_bars(coin, TimeFrame.Day, "2020-01-01", "2022-01-01", exchanges=["FTXU"]).df
  bars['daily_return'] = bars['close'].pct_change()  
  cumulative_return = bars['daily_return'].add(1).cumprod().sub(1).dropna()
  returns[coin] = cumulative_return

df = pd.DataFrame(returns)

df['Balanced ETF'] = df.mean(axis=1)

df.plot()

In [ ]:
#Calculating Historical Returns For Strategy
# Crytpto coin symbol listings
coins = [

"AAVEUSD",   #	Aave (AAVE)
"BATUSD",    #	Basic Attention Token (BAT)
"BTCUSD",    #	Bitcoin (BTC)
"BCHUSD",    #	Bitcoin Cash (BCH)
"LINKUSD",   #	ChainLink Token (LINK)
"DAIUSD",    #	Dai (DAI)
"DOGEUSD",   #	Dogecoin (DOGE)
"ETHUSD",    #	Ethereum (ETH)
"GRTUSD",    #	Graph Token (GRT)
"LTCUSD",    #	Litecoin (LTC)
"MKRUSD",    #	Maker (MKR)
"MATICUSD",  #	Matic (MATIC)
"PAXGUSD",   #	PAX Gold (PAXG)
"SHIBUSD",   #	Shiba Inu (SHIB)
"SOLUSD",    #	Solana (SOL)
"SUSHIUSD",  #	Sushi (SUSHI)
"USDTUSD",   #	Tether (USDT)
"TRXUSD",    #	TRON (TRX)
"UNIUSD",    #	Uniswap Protocol Token (UNI)
"WBTCUSD",   # 	Wrapped BTC (WBTC)
"YFIUSD",    #  Yearn.Finance (YFI)

]

In [ ]:
#Comparing Strategies Using The Sharpe Ratio
risk_free_rate = 0.02
sharpe_ratio = (df.iloc[-1] - risk_free_rate) / df.std()

# sort in descending order
sharpe_ratio.sort_values(ascending=False)

In [ ]:
# Creating Our Own Balanced Crypto ETF 
def rebalance_crypto_etf():
    account = alpaca_client.get_account()

	# delegate 95% of portfolio to ETF, leaving 5% cash on hand 
    # allows us to account for slippage or changes in margin requirements
    buying_power = float(account.portfolio_value) * 0.95
	
    # the rebalance threshold is the amount an allocation must deviate
    # before we place an order to rebalance it
    rebalance_threshold = 0.001 * buying_power
	
    # the desired allocation per coin
    desired_allocation = buying_power / len(coins)

    for coin in coins:
      
      # the current position size of that coin in our portfolio
      current_value = get_position_value(coin)
      
      # the difference between the desired allocation and existing allocation
      delta_alloc = desired_allocation - current_value
      
      # buy if delta is positive else sell
      side = "buy" if delta_alloc > 0 else "sell"
      
      notional = abs(delta_alloc)
      
      if notional > rebalance_threshold:
        alpaca_client.submit_order(symbol=coin, side=side, notional=notional)


def get_position_value(symbol):
	"""Returns the market value of holdings in a given symbol."""
    try:
        position = alpaca_client.get_position(symbol)
        return float(position.market_value)
    except:
		# if there is no position open, a 404 error is thrown
        # 0 market value for that symbol
        return 0


In [ ]:
#Scheduling For Rebalance 
# [pip install schedule] 
import schedule
import time

schedule.every().monday.at("10:30").do(rebalance_crypto_etf)

while True:
    schedule.run_pending()
    time.sleep(1)